In [1]:
import nltk
from os import getcwd
#import w1_unittest

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/emmabarnes/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emmabarnes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
filepath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filepath)

In [13]:
pip install nltk


[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install utils


[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples


In [4]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from nltk.tokenize import TweetTokenizer


In [5]:
import re
import string

In [13]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation): 
        # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)
    return tweets_clean

In [7]:
def build_freqs(tweets, ys):
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [8]:
all_pos = twitter_samples.strings('positive_tweets.json')
all_neg = twitter_samples.strings('negative_tweets.json')

In [36]:
print(type(twitter_samples))

<class 'nltk.corpus.reader.twitter.TwitterCorpusReader'>


In [38]:
print(type(all_pos))

<class 'list'>


In [9]:
test_pos = all_pos[4000:]
train_pos = all_pos[:4000]
test_neg = all_neg[4000:]
train_neg = all_neg[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [40]:
print(type(train_x[0]))

<class 'str'>


 - numpy array of all positive and negative labels

In [10]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)),axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)),axis=0)

In [11]:
#test split should be 20% train set should be 80%

print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


 - create frequency dictionary with the build_freqs thing
 - the outer loop goes through each text body, the inner loop through each word in the text
 - the key is the tuple (word, label) such as ("happy", 1) or ("happy", 0). Value stored for each key is the count of how many times the word happy gets associated with a positive/negative label

In [14]:
freqs = build_freqs(train_x, train_y)

In [15]:
print("type freqs = " + str(type(freqs)))
print("length freqs = " + str(len(freqs.keys())))

type freqs = <class 'dict'>
length freqs = 11338


In [16]:
print(train_x[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [17]:
print(process_tweet(train_x[0]))

['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


 - sigmoid maps z to a value ranging between 0 and 1, so it can be treated as a probability
 - this function should work if z is a scalar or an array representing a vector

In [18]:
def sigmoid(z):
    h = 1/(1+np.exp(-z))
    return h

 - the logic of logistic regression is taking a regular linear regression and applying a sigmoid to the output
 - regression: 
     z = theta0X0 + theta1X1 + ... + thetaNXN
     theta - weights (w vector)
 - logistic regression:
     h(z) = 1/1+exp^(-z)

 The cost function used for log reg is the average of the log loss across all the training examples: function J(theta)
 - where m is the number of training examples
 - y^(i) is the actual label of training example 'i'
 - h(z^(i)) is the model's prediction for the training example 'i'
 
 the loss function Loss for a single training example is:
     Loss = -1 X (y^(i)log(h(z(theta)^i))+(1-y^i)log(1-h(z(theta)^i)))
 - all the h values are between 0 and 1, so logs will be negative. This is why we apply factor of -1 to the sum of the two loss terms
 - when the model predicts 1 (h(z(theta)) = 1) and the label 'y' is also 1, the loss for that training example is 0. 
 - when the model predicts 0 (h(z(theta)) = 0) and the actual label is also 0, the loss for that training example is 0.
 - When the model prediction is close to 1 (h(z(theta)) = 0.9999) and the label is 0, the second term of the log loss becomes a large negative number which is multiplied by the overall factor of -1 to convert it to a positive loss value. The closer the model prediction gets to 1, the greater the loss. 

To update weight/parameter vector theta, apply gradient descent to iteratively improve model predictions by weight. 
The gradient of the cost function J with respect to one weight theta(sub j).
 - i is the index across all m training examples
 - j is the index of the weight theta sub j ,so that x^i sub j is the feature associated with the weight theta sub j
 - to update the weight theta sub j, adjust by subtracting a fraction of the gradient determined by alpha. 
 theta(sub j) = theta(sub j) - alpha X delta(sub theta (sub j))J(theta)
 - the LEARNING RATE alpha is a value WE CHOOSE to control how big a single update will be for each pass. 

For the gradient descent function...

 - the number of iterations num_iters is the number of times you'll use the entire training set
 - for each iteration, calculate the cost function using all training examples ('m' training examples), and for all features
 - instead of updating single weight theta(sub i) at a time, apply updates to the weights in the column vector theta for all theta(sub n)
 
 - theta has dimensions(n+1, 1) where 'n' is the number of features, and there is one more element for the bias term theta(sub 0) the corresponding feature value x(sub 0) is just 1. 
 - The z are calculated by multiplying the feature matrix 'x' with the weight vector 'theta' z = x*theta
     - x has dimensions (m, n+1)
     - theta has dimensions (n+1, 1)
     - z has dimensions (m, 1)
 - The prediction h is calculated by applying the sigmoid to each element in 'z': h(z) = sigmoid(z), and has dimensions (m, 1)
 - the cost function J is calculated by taking the dot product of the vectors 'y' and 'log(h)'. Since both y and h are column vectors of size (m, 1), transpose the vector to the LEFT so that the matrix multiplication of a row vector with a column vector handles the dot product. (T in my equations are transpose, i'm doing vector stuff :D)
 - the update of theta is also vectorized. Since dimensions of x are (m, n+1), and both h and y are (m, 1), we need to transpose the x and move it left to preform the matrix multiplication wiht yields (n+1, 1) result. 

In [19]:
def gradient (x, y, theta, alpha, num_iters):
    m = len(y)
    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)
        J = (-1/m)*np.sum(y*np.log(h)+(1-y)*np.log(1-h))
        theta = theta-(alpha/m)*np.dot(x.T, (h-y))
    J = float(J)
    return J, theta

In [21]:
#check
np.random.seed(1)
#X input is 10 by 3 dim with 1s for biases
tmp_X = np.append(np.ones((10,1)), np.random.rand(10,2)*2000, axis=1)
#Y labels are 10 by 1
tmp_Y = (np.random.rand(10,1)>0.35).astype(float)

#with gradient
tmp_J, tmp_theta = gradient(tmp_X, tmp_Y, np.zeros((3,1)), 1e-8, 700)
print("cost" + str(tmp_J))
print("weight vector " + str([round(t, 8) for t in np.squeeze(tmp_theta)]))


cost0.6709497038162117
weight vector [4.1e-07, 0.00035658, 7.309e-05]


 - given a list of reviews/text extract the features and put them in a matrix, extract two 
     - positive (illicit)
     - negative (non illicit)
 - train logistic classifier on these features
 - test on the validation set

 - function takes a single text body
 - processes text
 - loops through each word of processed words
     - for each word, check freqs dictionary for the count when that word has a positive '1' label(check for key word (word, 1.0
     - do the same for when the word is associated wiht negative label 0 
NOTE: the prediciton being positive or negative depends on feature vector which counts-in duplicate words 

In [22]:
def extractFeatures(tweet, freqs, process_tweet=process_tweet):
    word_l = process_tweet(tweet)
    x = np.zeros(3)
    x[0] = 1
    
    for word in word_l:
        x[1] += freqs.get((word, 1), 0) #positive label
        x[2] += freqs.get((word, 0), 0) #neg label
    x = x[None, :]
    assert(x.shape == (1,3))
    return x

In [23]:
#first test on training data
tmp1 = extractFeatures(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


to train the model...
 - stack the features for all training examples into a matrix X
 - call the gradient function to update weights

In [26]:
X = np.zeros((len(train_x), 3)) #collects features x and stacks to matrix X

for i in range(len(train_x)):
    X[i, :]= extractFeatures(train_x[i], freqs)
    
Y = train_y #training labels for X

#apply gradient
J, theta = gradient(X, Y, np.zeros((3, 1)), 1e-9, 1500)

print("cost of training" + str(J))
print("vector weights" + str([round(t, 8) for t in np.squeeze(theta)]))



cost of training0.24215473924831094
vector weights[7e-08, 0.00052391, -0.00055517]


In [27]:
def predict(tweet, freqs, theta):
    x = extractFeatures(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))
    
    return y_pred

In [28]:
#testing output
for tweet in ['I am happy', 'I am sad', 'this sucks', 'this is great', 'great', 'great great']:
    print(str(tweet) + " === " + str(predict(tweet, freqs, theta)))

I am happy === [[0.51858054]]
I am sad === [[0.48677862]]
this sucks === [[0.49827197]]
this is great === [[0.51546416]]
great === [[0.51546416]]
great great === [[0.53089874]]


After training the model check how it will work on unseen data by testing against test set

 - given test data and weights of trained mode, calculate accuracy
 - use predict to make predictions on each in the test set
 - if the prediction is >0.5 set the model's classification y_hat to 1, otherwise 0.
 - the prediction is accurate when y_hat = test_y. Sum the cases when this happens and divide by m to get average

In [31]:
def testAccuracy (test_x, test_y, freqs, theta, predict=predict):
    y_hat = []
    for tweet in test_x:
        y_pred = predict(tweet, freqs, theta)
        
        if y_pred > 0.5: #good prediction
            y_hat.append(1.0)
        else:
            y_hat.append(0.0)
            
    accuracy = np.sum(y_hat==test_y.squeeze())/len(test_x)#merge into a 1d array with == to compare
    
    return accuracy

In [33]:
tmp_accuracy = testAccuracy(test_x, test_y, freqs, theta)
print("accuracy: " + str(tmp_accuracy))

accuracy: 0.995


In [35]:
test = 'this is a really good thing. I loved this thing but the ending was terrible..'
print(process_tweet(test))
y_hat = predict(test, freqs, theta)
print(y_hat)
if y_hat>0.5:
    print("POSITIVE")
else:
    print("NEGATIVE")

['realli', 'good', 'thing', 'love', 'thing', 'end', 'terribl', '..']
[[0.53101143]]
POSITIVE
